In [ ]:
# Colab 安裝 pyngrok
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=042bf4bcce3a350456e8c0f8a65dde45c49eb73fb3e060ec3276a57242ad121c
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [ ]:
# 安裝 line-bot-sdk
!pip install line-bot-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
# 安裝 watchdog
!pip install watchdog

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 2.1 MB/s eta 0:00:00


In [ ]:
# 查看本機安裝的 line-bot-sdk 資訊
!pip show watchdog

Name: watchdog
Version: 2.2.1
Summary: Filesystem events monitoring
Home-page: https://github.com/gorakhargosh/watchdog
Author: Yesudeep Mangalapilly
Author-email: yesudeep@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: 
Required-by: 


In [ ]:
# 引入 flask 模組中的 Flask, request, abort 物件
from flask import Flask, request, abort

# 引入 linebot 模組中的 LineBotApi（Line Token）, WebhookHandler（Line Secret）
from linebot import (
    LineBotApi, WebhookHandler
)

# 引入 linebot.exceptions 模組中的 InvalidSignatureError（錯誤偵錯的）
from linebot.exceptions import (
    InvalidSignatureError
)

# 引入 linebot.models 模組中的 MessageEvent, TextMessage, TextSendMessage
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# 檔案監聽觀察者
from watchdog.observers import Observer

# 檔案監聽事件處理者
from watchdog.events import FileSystemEventHandler

# 系統路徑套件（判斷檔案是否存在）
import os.path

# 執行緒模組（用來配合檔案監聽觀察者，確認檔案是否有變動）
import threading

# 時間模組（每1秒監聽一次檔案是否有變動）
import time

# 引入 pyngrok 模組中的 ngrok 物件
from pyngrok import ngrok

# 引入 pandas 模組並取名為 pd
import pandas as pd

# 引入 tabulate 模組中的 tabulate 物件
from tabulate import tabulate

################################################################################

# __name__ 模組名稱
app = Flask(__name__)
# line bot - Channel access token
line_bot_api = LineBotApi('你的LineBotApi')
# line bot - Channel secret
handler = WebhookHandler('你的handler')
# 監聽檔案執行緒開關
done = False
# 檔案路徑
file_path = '/content/drive/MyDrive/Colab/Data/break_high.csv'

################################################################################

# 讀檔函式
def checkZ_data():
    # skiprows 跳過第幾列不讀, header 檔案標頭, encoding 文字編碼格式
    data_csv = pd.read_csv(file_path, skiprows=[0], header=None, encoding='utf-8')
    # 將讀入的資料轉型成 list
    data_list = list(data_csv[3])

    # 紀錄異常次數
    error_count = 0

    # 遍歷所有資料判斷異常
    for i in data_list:
        # 異常值判斷（大於 1.6 或 小於 -2）
        if i > 1.6 or i < -2:
            error_count = error_count + 1

    return str(error_count)

def checkZ_error():
    # skiprows 跳過第幾列不讀, header 檔案標頭, encoding 文字編碼格式
    data_csv = pd.read_csv(file_path, skiprows=[0], header=None, encoding='utf-8')
    # 將讀入的資料轉型成 list
    z_data = list(data_csv[3])

    # 紀錄異常次數
    error_list = []

    # 遍歷所有資料判斷異常
    for i in z_data:
        # 異常值判斷（大於 1.6 或 小於 -2）
        if i > 1.6 or i < -1.9:
            error_list.append([i])

    ## tbulate
    table = tabulate(error_list, headers=['Z'], tablefmt='simple')
    return str(table)

def first10_data():
    # skiprows 跳過第幾列不讀, header 檔案標頭, encoding 文字編碼格式
    data_csv = pd.read_csv(file_path, skiprows=[0], header=None, encoding='utf-8')
    # 將讀入的資料轉型成 list
    x_data = list(data_csv[1])
    y_data = list(data_csv[2])
    z_data = list(data_csv[3])

    # 紀錄前 10 筆
    first10 = []

    # 遍歷前 10 筆資料
    for i in range(0, 10):
        first10.append([x_data[i], y_data[i], z_data[i]])

    # 轉換成表格
    table = tabulate(first10, headers=['x', 'y', 'z'], tablefmt='simple')
    return str(table)

def first20_data():
    # skiprows 跳過第幾列不讀, header 檔案標頭, encoding 文字編碼格式
    data_csv = pd.read_csv(file_path, skiprows=[0], header=None, encoding='utf-8')
    # 將讀入的資料轉型成 list
    x_data = list(data_csv[1])
    y_data = list(data_csv[2])
    z_data = list(data_csv[3])

    # 紀錄前 20 筆
    first20 = []

    # 遍歷前 20 筆資料
    for i in range(0, 20):
        first20.append([x_data[i], y_data[i], z_data[i]])

    # 轉換成表格
    table = tabulate(first20, headers=['x', 'y', 'z'], tablefmt='simple')

    return str(table)

def newest100_data():
    # skiprows 跳過第幾列不讀, header 檔案標頭, encoding 文字編碼格式
    data_csv = pd.read_csv(file_path, skiprows=[0], header=None, encoding='utf-8')
    # 將讀入的資料轉型成 list
    x_data = list(data_csv[1][-100:])
    y_data = list(data_csv[2][-100:])
    z_data = list(data_csv[3][-100:])

    # 紀錄前 20 筆
    first20 = []

    # 遍歷前 20 筆資料
    for i in range(0, 100):
        # x 平方 + y 平方 + z 平方
        pow_value = x_data[i] ** 2 + y_data[i] ** 2 + z_data[i] ** 2
        # 取小數點到第二位 round(值, 2)
        first20.append([x_data[i], y_data[i], z_data[i], round(pow_value, 2)])

    # 轉換成表格
    table = tabulate(first20, headers=['x', 'y', 'z', '**'], tablefmt='simple')

    return str(table)

def delete_file():
    print('delete')
    # 刪除檔案
    os.remove(file_path)
    # 刪除後檢查是否有刪除成功
    if not os.path.isfile(file_path):
        print('delete done')
        # 成功
        return '檔案刪除成功'
    else:
        # 失敗
        return '檔案刪除失敗'

# 檔案監聽函式
def file_listener():
    file_handler = MyEventHandler()
    path = '/content/drive/MyDrive/Colab/Data/'
    observer = Observer()
    observer.schedule(file_handler, path)
    observer.start()
    print('Monitoring')
    try:
        while not done:
            time.sleep(1)
    finally:
        observer.stop()
        observer.join()

# ngrok 函式
def conn_ngrok():
    # 設定 ngrok token
    ngrok.set_auth_token('YOUR_NGROK_TOKEN')

    # 通訊埠，埠號介於 0 與 65535 之間
    port = 5000

    # 將本地端網址(http://127.0.0.1) ngrok連線的
    public_url = ngrok.connect(port, bind_tls=True).public_url

    print(f'ngrok tunnel { public_url } -> local tunnel http://127.0.0.1:{ port }')

################################################################################

# 監聽檔案變動的物件
class MyEventHandler(FileSystemEventHandler):
    def on_moved(self, event):
        pass

    def on_created(self, event):
        if os.path.isfile(file_path):
            print('file exist')
            # 取得資料異常筆數
            error_count = checkZ_data()
            msg = 'Z 軸資料有' + error_count + '筆異常'
            line_bot_api.push_message('YOUR_LINE_USER_ID'
                , TextSendMessage(text=msg))

            # 取得異常資料
            msg = checkZ_error()
            line_bot_api.push_message('YOUR_LINE_USER_ID'
                , TextSendMessage(text=msg))
            # 推播完立即刪除檔案
            # delete_file()

    def on_deleted(self, event):
        pass

    def on_modified(self, event):
        pass

################################################################################

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    # 當開發者的Webhook伺服器收到以POST方式所傳送的LINE事件訊息時，必須要立即驗證該事件訊息是否真的來自LINE平台，以避免被偽造的訊息所欺騙造成資訊安全危機
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# line bot 訊息接收處
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 使用者傳過來的訊息
    user_message = event.message.text

    # 依據使用者傳過來的訊息回應
    if os.path.isfile(file_path):
        print('file exist')
        msg = ''
        if user_message == 'checkZ':
            error_count = checkZ_data()
            msg = 'Z 軸資料有' + error_count + '筆異常'
        elif user_message == 'errorZ':
            msg = checkZ_error()
        elif user_message == 'first10':
            msg = first10_data()
        elif user_message == 'first20':
            msg = first20_data()
        elif user_message == 'delete':
            msg = delete_file()

        if not (msg == ''):
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=msg))

################################################################################

if __name__ == "__main__":
    # 呼叫 conn_ngrok() 函式
    conn_ngrok()
    # 檔案監聽器執行緒
    file_thread = threading.Thread(target=file_listener)
    # 啟動檔案監聽執行緒
    file_thread.start()
    app.run(port=4455)
    # 告訴檔案監聽器結束
    done = True

ERROR:pyngrok.process.ngrok:t=2023-02-08T03:52:12+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_NGROK_TOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: ignored